### Importing necessary libs

In [72]:
import os
import string
from nltk import tokenize
from gensim.models import Word2Vec
from nltk import word_tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
import gensim
from nltk.corpus import stopwords
import sentence_transformers
from sentence_transformers import SentenceTransformer,util
from sklearn.cluster import KMeans
import re
import statistics
from keras.layers import Average

### defining the model with the pretrainded data

In [73]:
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")
# embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')


### structring words

In [74]:
listStructureWords =  []

df = pd.read_excel("structureWords.xlsx")
listStructureWords = df.iloc[:,0].tolist()

### Adding only 100 Essays for embeddings

In [75]:
stopWords = ["directed toward young children", "from two to five","two to five", "to young children", "directed toward"]
regex = r"\b(?:{})\b".format("|".join(listStructureWords))

sentences = []
j=0
df_fileName = pd.DataFrame({})
for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\10 essays for essays reference"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\10 essays for essays reference", filename)) as f:

       text = f.read()
       text = text.replace("ï»¿","")
       for st in stopWords:
           text = text.replace(st,"")
       # for w in tokenize.word_tokenize(text):
       #     if w in stopWords:
       #         text = text.replace(w,"");
       #sents = tokenize.sent_tokenize(text)
       sents = re.split(regex, text)
       #sents = text.split("\n\n")
       for s in sents:
           # if (para.isspace()):
           #     continue
               #print("The string is empty")
           #print(para)
           #print("###############")
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           s = s.translate(str.maketrans(' ', ' ', string.punctuation))
           if (len(s.split()) <5):
               continue
           sentences.append(s)
           df_fileName = df_fileName.append({
             "ios" : j,
             "file name": filename,
              "argument": s
              },ignore_index=True)
           j = j+1


#df_fileName.to_csv("FileName with the senteces.csv")

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_6272\233329389.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fileName = df_fileName.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_6272\233329389.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fileName = df_fileName.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_6272\233329389.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fileName = df_fileName.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_6272\233329389.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fileName = df_fileName.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_6272\2333293

### Getting all the paragraphs inside the essays(Bad method)

In [ ]:
#indexs =0
paragraph_num=[]
for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2"):
    y = open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2", filename), 'r')
    paragraph = 0
    lines = y.readlines()
    for idx, line in enumerate(lines):
        if not line == '\n':
            m = re.search(r'\w', line)
            if m is None:
                continue
            str = m.group(0)
        try:
            if line == '\n' and str in lines[idx-1]:
                paragraph +=1
        except:
            pass
    if lines[-1] != '\n': # if the last line is not a new line, count a paragraph.
        paragraph +=1
    #indexs = indexs+1
    #print("The append starting" ,paragraph, indexs)
    #print("############################")
    if (paragraph >2):
        paragraph_num.append(paragraph)

average = statistics.mean(paragraph_num)
#print(indexs)
print(average)
print(round(average))
print(len(paragraph_num))

### Saving to CSV file

In [76]:
df_fileName.to_csv("FileName with the senteces.csv")

### Getting the embeddings

In [77]:
corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

corpus_embeddings.shape

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

(65, 768)

### Testing torch with cuda

In [78]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

### applying the clustering method

In [79]:
k = 10

clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "cluster": cluster_labels
})

For n_clusters = 10
Silhouette coefficient: 0.08
Inertia:24.390676498413086
Silhouette values:
    Cluster 2: Size:18 | Avg:0.16 | Min:0.05 | Max: 0.27
    Cluster 6: Size:10 | Avg:0.09 | Min:-0.01 | Max: 0.18
    Cluster 7: Size:2 | Avg:0.07 | Min:0.03 | Max: 0.10
    Cluster 1: Size:20 | Avg:0.04 | Min:-0.07 | Max: 0.13
    Cluster 3: Size:3 | Avg:0.03 | Min:0.01 | Max: 0.06
    Cluster 4: Size:5 | Avg:0.02 | Min:-0.02 | Max: 0.07
    Cluster 0: Size:1 | Avg:0.00 | Min:0.00 | Max: 0.00
    Cluster 8: Size:1 | Avg:0.00 | Min:0.00 | Max: 0.00
    Cluster 9: Size:1 | Avg:0.00 | Min:0.00 | Max: 0.00
    Cluster 5: Size:4 | Avg:-0.01 | Min:-0.12 | Max: 0.06


### random statments from cluster

In [56]:
test_cluster = 0
queryCount = len(df_clusters.query(f"cluster == {test_cluster}"))

print(queryCount)

if (queryCount > 10):
    for i,t in enumerate(df_clusters.query(f"cluster == {test_cluster}").sample(10).iterrows()):
        print(t[1]["text"])
        print("-------------")

else:
    for i,t in enumerate(df_clusters.query(f"cluster == {test_cluster}").sample(queryCount).iterrows()):
        print(t[1]["text"])
        print("-------------")


1149
i am writing an essay about the question if it is good 
-------------
 it comes to what foods they eat after all how many adverts for fruit roll ups and sugary juice boxes can we all remember being streamed into our conciousness as young children and infdluencing what we twanted to eat
i 
-------------
maybe 
yes and no         hohoho
because of reasons                        lã¶lã¶lã¶lã¶lã¶lã¶lã¶lã¶lã¶lã¶lã¶lã¶
there are adventages and disadventages about this
ubi est quintus caecilia clamat
quintus in hortus est
nihin mater nihil quintus sedet et tacet
ein essay welches diese schreibaufgabe gut erfã¼llt umfasst mindestens 300 wã¶rter ihr text wird anhand unterschiedlicher qualitã¤ten beurteilt diese beinhalten die darlegung ihrer ideen die organisation des essays sowie die sprachliche qualitã¤t und genauigkeit sie haben dafã¼r 30 minuten zeit versuchen sie die komplette zeit 
-------------
 it is easier for parents to controle there children in the public or at home 
-----------

### Most representative clusters

In [80]:
test_cluster = 0
most_representative_docs = np.argsort(
    np.linalg.norm(corpus_embeddings - clustering.cluster_centers_[test_cluster], axis=1)
)
# print(most_representative_docs[0])

if (queryCount > 10):
    for d in most_representative_docs[:10]:
        print(d)
        print(sentences[d])
        print("-------------")
else:
    for d in most_representative_docs[:queryCount]:
        print(d)
        print(sentences[d])
        print("-------------")


38
 reason is that young children at the age  can not understand the main statement from the television because there brain can not translate or understand the film which is shown as a 
-------------
44
 on their there future lives could be mainly destroyed or would stay on the lies of the television my last reason is that children at the age of  are defintiv to young to look tv because of the risks and the 
-------------
64
 with this statement because it is not bad if little child seeing television advertising in this age but they should not watch it 
-------------
40
 reason is that this kids should not grow up with a television because if they look tv at the begin of their lives they could focus their future to look fluently and hours 
-------------
42
 is that the television is a media which is lying fluently to have better quotes of the public to become famous and rich the children would grow up with wrong information 
-------------
30
 they shopuld have a time where they are all

### Saving to .csv File

In [81]:
df_clusters.to_csv(f"SbertClustering-All-Essays- {k} -Clusters.csv")

### saving most representative to csv file

In [82]:
df_mostRepresentative = pd.DataFrame({})
df_ClsuteringSentencesCount = pd.DataFrame({})

for i in range (k) :
    listSentencesMostRep =[]
    most_representative_docs_Save = np.argsort(
    np.linalg.norm(corpus_embeddings - clustering.cluster_centers_[i], axis=1)
    )
    queryMostCount = len(df_clusters.query(f"cluster == {i}"))
    df_ClsuteringSentencesCount = df_ClsuteringSentencesCount.append({
       "cluster" : i,
        "Sentences Count": queryMostCount
    },ignore_index=True)
    if (queryMostCount > 10):
        for d in most_representative_docs_Save[:10]:
            listSentencesMostRep.append(sentences[d])
    else:
        for d in most_representative_docs_Save[:queryMostCount]:
            listSentencesMostRep.append(sentences[d])

    for sent in listSentencesMostRep:
        df_mostRepresentative = df_mostRepresentative.append({
         "text": sent,
         "cluster": i
         }, ignore_index=True)



df_mostRepresentative.to_csv(f"SbertClustering-All-Essays- {k} -Clusters - 10MostRepresentative.csv")
df_ClsuteringSentencesCount.to_csv(f"SbertClustering-All-Essays- {k} - Clusters - ClusteringCount.csv")

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_6272\4001967623.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ClsuteringSentencesCount = df_ClsuteringSentencesCount.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_6272\4001967623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mostRepresentative = df_mostRepresentative.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_6272\4001967623.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ClsuteringSentencesCount = df_ClsuteringSentencesCount.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_6272\4001967623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_

### Test

In [ ]:
listTest = ["Ahmed" , "mido", "test test"]
listNumbers = [1 , 10, 15]
df_mostRepresentative = pd.DataFrame({
})

df_mostRepresentative = df_mostRepresentative.append({
    "text": listTest,
    "cluster": listNumbers
}, ignore_index=True)

listTest = ["After" , "after mido", "test after"]
listNumbers = [2 , 3, 65]

df_mostRepresentative = df_mostRepresentative.append({
    "text": listTest,
    "cluster": listNumbers
}, ignore_index=True)

df_mostRepresentative.to_csv('test.csv')

### Testing torch with cuda


In [39]:
import torch

torch.cuda.is_available()

True

### Testing word_embeddings

In [ ]:
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'A man is eating pasta.',
          'The girl is carrying a baby.',
          'The baby is carried by the woman',
          'A man is riding a horse.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.',
          'A cheetah is running behind its prey.',
          'A cheetah chases prey on across a field.'
          ]
corpus_embeddings = model.encode(corpus)

########################## TESTING #########################################################
#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(corpus_embeddings, corpus_embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(corpus[i], corpus[j], cos_sim[i][j]))

for sentence, embedding in zip(corpus, corpus_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")